In [0]:
from pyspark.sql.functions import to_date, col, round, current_timestamp
import sys
from time import time
from delta.tables import DeltaTable

In [0]:
# sys.path.append("/Workspace/Users/faraaz_khan@epam.com/PEI/utils")

In [0]:
# from utils.common_util 

In [0]:
start_time = time()

In [0]:
df = spark.read.table("fz_catalog.bronze.customers").select(
    "customer_id", "customer_name", "country"
)

In [0]:
df_silver = df.withColumn("created_date", current_timestamp())

In [0]:
df_silver.write.mode("overwrite").saveAsTable("fz_catalog.silver.customers")

In [0]:
end_time = time()

In [0]:
dbutils.notebook.exit(f"Run Time: {end_time - start_time} seconds")